In [56]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
os.chdir("D:\\GoogleDrive\\InterviewAssignment\\GoJekDriverAcceptanceGit\\GoJekAssignment")

df_train  = pd.read_csv("data\\processed\\transformed_dataset_train.csv")
df_test  = pd.read_csv("data\\processed\\transformed_dataset_test.csv")


features=["driver_gps_accuracy",
    "trip_distance",
    "hourofday",
    "hourofday_average",
    "driver_hourofday_average",
    "dayofweek",
    "dayofweek_average",
    "driver_dayofweek_average",
    "is_accepted_last5day_rolling_mean"
    # "cluster_driver_labels",
    # "cluster_customer_labels"
    ]

param_grid_big = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,8,16,32],
    'criterion' :['entropy'],
}

#Taking single value bcs of computational resource issue 
param_grid = { 
    'n_estimators': [200],
    'max_features': ['auto'],
    'max_depth' : [8,16],
    'criterion' :['entropy'],
}

rf_estimator = RandomForestClassifier(n_jobs=-1,random_state=33,bootstrap=False,class_weight="balanced")
CV_rfc = GridSearchCV(estimator=rf_estimator, param_grid=param_grid, cv= 2)
CV_rfc.fit(df_train[features],df_train['is_accepted'])

GridSearchCV(cv=2,
             estimator=RandomForestClassifier(bootstrap=False,
                                              class_weight='balanced',
                                              n_jobs=-1, random_state=33),
             param_grid={'criterion': ['entropy'], 'max_depth': [8, 16],
                         'max_features': ['auto'], 'n_estimators': [200]})

In [57]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 16,
 'max_features': 'auto',
 'n_estimators': 200}

In [58]:
dict(zip(features, CV_rfc.best_estimator_.feature_importances_))

{'driver_gps_accuracy': 0.006100602485373832,
 'trip_distance': 0.021634242975308592,
 'hourofday': 0.0030870522845956386,
 'hourofday_average': 0.0037435928609502165,
 'driver_hourofday_average': 0.53128867614759,
 'dayofweek': 0.0017840452529426889,
 'dayofweek_average': 0.0018621647342080894,
 'driver_dayofweek_average': 0.35379902083842857,
 'is_accepted_last5day_rolling_mean': 0.07670060242060238}

In [59]:
from sklearn.metrics import f1_score
y_true=df_test["is_accepted"].values
y_pred=CV_rfc.best_estimator_.predict(df_test[features].values)
metrics = {}
metrics["f_score_class_1"]=f1_score(y_true,y_pred,pos_label=1)
metrics["f_score_class_0"]=f1_score(y_true,y_pred,pos_label=0)

In [60]:
metrics

{'f_score_class_1': 0.9117126730504619, 'f_score_class_0': 0.2633552014995314}

In [61]:
import numpy as np 
np.unique(y_true, return_counts=True)

(array([0, 1], dtype=int64), array([ 3846, 36033], dtype=int64))